In [1]:
import math
import numpy as np

def safe_log(x):
    return float('-inf') if x == 0 else math.log(x)

def calculate_path_log_probability(path, sequence, start_probs, trans_probs, emit_probs):
    if len(path) != len(sequence):
        raise ValueError("Path and sequence must have the same length")
    
    log_prob = 0.0
    
    for i in range(len(sequence)):
        current_state = path[i]
        current_symbol = sequence[i]
        if i == 0:
            log_prob += safe_log(start_probs[current_state])
        else:
            prev_state = path[i-1]
            log_prob += safe_log(trans_probs[prev_state][current_state])
        
        log_prob += safe_log(emit_probs[current_state][current_symbol])
    
    final_state = path[-1]
    if final_state == 'I' and 'end' in trans_probs['I']:
        log_prob += safe_log(trans_probs['I']['end'])
    
    return log_prob

hmm_states = ['E', '5', 'I']

hmm_start_probs = {
    'E': 1.0,   
    '5': 0.0,   
    'I': 0.0    
}

hmm_trans_probs = {
    'E': {'E': 0.9, '5': 0.1},          
    '5': {'I': 1.0},                   
    'I': {'I': 0.9, 'end': 0.1}        
}

hmm_emit_probs = {
    'E': {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25},  
    '5': {'A': 0.05, 'C': 0.0, 'G': 0.95, 'T': 0.0},   
    'I': {'A': 0.4, 'C': 0.1, 'G': 0.1, 'T': 0.4}      
}

example_path = "EEEEEEEEEEEEEEEEEE5IIIIIII"
example_sequence = "CTTCATGTGAAAGCAGACGTAAGTCA"

log_prob = calculate_path_log_probability(
    example_path, 
    example_sequence, 
    hmm_start_probs, 
    hmm_trans_probs, 
    hmm_emit_probs
)

print(f"Log probability of the given path: {log_prob:.2f}")

Log probability of the given path: -41.22


In [ ]:
## second part

In [5]:
import math

# Define states and observations
states = ['E','5', 'I']
observations = ['A', 'C', 'G', 'T']

# Start probabilities
start_p = {'E': 1,'5':0,'I':0}

# Transition probabilities
trans_p = {
    'E': {'E': 0.9, '5': 0.1, 'I': 0},
    '5': {'E': 0, '5': 0, 'I': 1},
    'I': {'E': 0, '5': 0, 'I': 1}
}


# Emission probabilities
emit_p = {
    'E': {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25},
    'I': {'A': 0.4,  'C': 0.1,  'G': 0.1,  'T': 0.4},
    '5': {'A': 0.05, 'C': 0, 'G': 0.95, 'T': 0} 
}



def safe_log(x):
    return float('-inf') if x == 0 else math.log(x)

def viterbi(sequence, states, start_p, trans_p, emit_p):
    n = len(sequence)
    matrix = [{} for _ in range(n)]
    backpointer = [{} for _ in range(n)]
    path = []

    # Initialize
    for state in states:
        matrix[0][state] = safe_log(start_p[state]) + safe_log(emit_p[state][sequence[0]])
        backpointer[0][state] = None

    # Fill the matrix
    for i in range(1, n):
        for curr_state in states:
            max_val = float('-inf')
            max_state = None
            for prev_state in states:
                prob = matrix[i-1][prev_state] + safe_log(trans_p[prev_state][curr_state])
                if prob > max_val:
                    max_val = prob
                    max_state = prev_state
            matrix[i][curr_state] = max_val + safe_log(emit_p[curr_state][sequence[i]])
            backpointer[i][curr_state] = max_state
            # print(max_state)
            

    # Find last best state
    last_probs = matrix[n-1]
    final_state = max(last_probs, key=last_probs.get)
    max_log_prob = last_probs[final_state]
    # print(backpointer)
    # Backtrack to get the path
    path = [final_state]
    for i in range(n-1, 0, -1):
        path.insert(0, backpointer[i][path[0]])

    return path, max_log_prob



sequence = "CTTCATGTGAAAGCAGACGTAAGTCA"
obs_seq = list(sequence)

most_likely_path, log_prob = viterbi(obs_seq, states, start_p, trans_p, emit_p)

print("Most likely path:")
print("".join(most_likely_path))
print("Log probability of the path:", log_prob)



Most likely path:
EEEEEEEEEEEEEEEEEE5IIIIIII
Log probability of the path: -38.284929499081535
